In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import os, sys
from google.colab import drive
drive.mount('/gdrive')
sys.path.insert(0, "/content/gdrive/My Drive/CS 189/hw/hw6-collabtry/")
%cd /gdrive/My\ Drive/CS\ 189/hw/hw6-collabtry/

In [0]:
import numpy as np
from torchsummary import summary
import pandas as pd
from sklearn.metrics import confusion_matrix
import pickle
import torch

In [0]:
!pip install torchsummary

In [0]:
np.random.seed(10)

In [0]:
models_data = []

In [0]:
def plot_loss(arch,train,val):
    mt = sum(train)/len(train)
    mv = sum(val)/len(val)
    plt.title(arch+", Avg Train Loss: "+str(round(mt,4))+", Avg Val Loss: "+str(round(mv,4)))
    plt.plot([i+1 for i in range(len(train))], train, 'r', label="train")
    plt.plot([i+1 for i in range(len(val))], val, 'b', label="validation")
    plt.xlabel("steps")
    plt.ylabel("loss")
    plt.legend()
    plt.show()
    
def data_track(data=None,save=True):
    if save and len(data) > 0:
        with open("data.txt", "wb") as file:   #Pickling
            pickle.dump(data, file)
        torch.cuda.empty_cache()
    else:
        torch.cuda.empty_cache()
        with open("data.txt", "rb") as file:   # Unpickling
            return pickle.load(file)

In [0]:
import torch
from torch.utils import data
import pandas as pd
import random
import json
import numpy as np
from skimage import io, transform
from PIL import Image

class Mds189(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, label_file, loader, transform):
        'Initialization'
        self.label_file = label_file
        self.loader = loader
        self.transform = transform
        self.label_map = ['reach','squat','pushup','inline',
                          'hamstrings','lunge','deadbug','stretch']
        self.data= pd.read_csv(self.label_file,header=None)

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)

    def map_label_to_int(self,y):
        'The labels need to be integers'
        label_map = {'reach_both': 0,        # the key frames are labeled with the side
                     'squat_both': 1,
                     'inline_left': 2,
                     'inline_right': 2,
                     'lunge_left': 3,
                     'lunge_right': 3,
                     'hamstrings_left': 4,
                     'hamstrings_right': 4,
                     'stretch_left': 5,
                     'stretch_right': 5,
                     'deadbug_left': 6,
                     'deadbug_right': 6,
                     'pushup_both': 7,
                     'reach': 0,            # the video frames don't have information about which side is moving 
                     'squat': 1,
                     'inline': 2,
                     'lunge': 3,
                     'hamstrings': 4,
                     'stretch': 5,
                     'deadbug': 6,
                     'pushup': 7,
                     'label': -1           # label is the placeholder in `videoframe_data_test.txt` for the kaggle frame labels
                    }
        return label_map[y]

    def __getitem__(self,idx):
        'Generates one sample of data'
        path,target = self.data.iloc[idx]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)
        movement = self.map_label_to_int(target)

        return sample,movement

In [0]:
import os, sys

import torch
import torch.nn as nn
from torch.utils import data
# from mds189 import Mds189
import numpy as np
from skimage import io, transform
import matplotlib.pyplot as plt

# import ipdb

import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision.models as models
from PIL import Image

# from savecsv import preds_to_csv
# from model import *

# Helper functions for loading images.
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)

def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)

# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#cudnn.benchmark = True


In [0]:
import numpy as np

def load_preprocess_data(is_train, is_key_frame, params):
    
    label_file_train, label_file_val, label_file_test = None, None, None
    test_dataset, test_loader = None, None
    
    if is_key_frame:
        label_file_train =  'dataloader_files/keyframe_data_train.txt'
        label_file_val  =  'dataloader_files/keyframe_data_val.txt'
        # NOTE: the kaggle competition test data is only for the video frames, not the key frames
        # this is why we don't have an equivalent label_file_test with keyframes
    else:
        label_file_train = 'dataloader_files/videoframe_data_train.txt'
        label_file_val = 'dataloader_files/videoframe_data_val.txt'
        label_file_test = 'dataloader_files/videoframe_data_test.txt'
        
    mean_keytrain = [134.010302198,118.599587912,102.038804945]
    std_keytrain = [23.5033438916,23.8827343458,24.5498666589]
    mean_randtrain = [133.714058398,118.396875912,102.262895484]
    std_randtrain = [23.2021839891,23.7064439547,24.3690056102]
    train_dataset = Mds189(label_file_train,loader=default_loader,\
                           transform=transforms.Compose([
                               transforms.ColorJitter(hue=.05, saturation=.05),
                               transforms.RandomHorizontalFlip(p=0.33),
                               transforms.RandomRotation(degrees=15),      
                               transforms.ToTensor(),
                               transforms.Normalize(mean_keytrain, std_keytrain)
                           ]))
    train_loader = data.DataLoader(train_dataset, **params)

    val_dataset = Mds189(label_file_val,loader=default_loader,\
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(mean_keytrain, std_keytrain)
                         ]))
    val_loader = data.DataLoader(val_dataset, **params)
    
    if not is_key_frame:
        test_dataset = Mds189(label_file_test,loader=default_loader,\
                              transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean_randtrain, std_randtrain)
                               ]))
        test_loader = data.DataLoader(test_dataset, **params)
    
    return train_loader, val_loader, test_loader

def modelling(model,is_train,is_key_frame,arch,params,hyperparams):
    train_loader, val_loader, test_loader = \
    load_preprocess_data(is_train,is_key_frame,params)
    train_losses, val_losses = train_val_model(model, 
                                               train_loader,
                                               val_loader, 
                                               hyperparams)
    return (train_losses, val_losses, arch, params)
    
def train_val_model(model, train_loader, val_loader, hyperparams):
    num_epochs,criterion,optimizer = \
    hyperparams["num_epochs"],\
    hyperparams["criterion"],\
    hyperparams["optimizer"]
    
    print('Beginning training...')
    total_step = len(train_loader)
    train_losses, val_losses = [], []
    for epoch in range(num_epochs):
        for i, (local_batch,local_labels) in enumerate(train_loader):
            local_ims, local_labels = local_batch.to(device), local_labels.to(device)
            outputs = model.forward(local_ims)
            loss = criterion(outputs, local_labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            if (i+1) % 4 == 0:
                torch.save(model.state_dict(), os.getcwd()+"/"+model.arch)
                print ('Epoch [{}/{}], Step [{}/{}], Training Loss: {:.4f}'
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
        for i, (local_batch,local_labels) in enumerate(val_loader):
            local_ims, local_labels = local_batch.to(device), local_labels.to(device)
            outputs = model.forward(local_ims)
            loss = criterion(outputs, local_labels)
            val_losses.append(loss.item())
        print('finished epoch {}, Training Loss: {:.4f}, Validation Loss: {:.4f}'
              .format(epoch+1, train_losses[epoch], val_losses[epoch]))
    torch.save(model.state_dict(), os.getcwd()+"/"+model.arch)
    return (train_losses, val_losses)

def get_accuracy(model, loader):
    print('Starting accuracy calculation!!!')
    with torch.no_grad():
      correct = 0
      total = 0
      predicted_list = []
      groundtruth_list = []
      for (local_batch,local_labels) in loader:
          # Transfer to GPU
          local_ims, local_labels = local_batch.to(device),\
          local_labels.to(device)
          outputs = model.forward(local_ims)
          _, predicted = torch.max(outputs.data, 1)
          total += local_labels.size(0)
          predicted_list.extend(predicted)
          groundtruth_list.extend(local_labels)
          correct += (predicted == local_labels).sum().item()
    
    predicted_list = [p.cpu().numpy().tolist() for p in predicted_list]
    groundtruth_list = [p.cpu().numpy().tolist() for p in groundtruth_list]
    acc = correct / total
    print("accuracy",acc)
    return acc,predicted_list,groundtruth_list

def move_val_acc(pl,gt):
    #use pl and gt to produce your confusion matrices
    # view the per-movement accuracy
    label_map = ['reach','squat','inline','lunge','hamstrings','stretch','deadbug','pushup']
    pairs = []
    for id in range(len(label_map)):
      num = float(sum([p and g for (p,g) in zip(np.array(pl)==np.array(gt),np.array(gt)==id)])+0.00)
      denom = float(sum(np.array(gt)==id)+0.0)
      acc = num/denom
      move = label_map[id]
      pairs.append((acc,move))
    return pairs

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNet(nn.Module):
    def __init__(self, arch):
        super(NeuralNet, self).__init__()
        self.arch = arch
        if self.arch[0:10] == "pytorch_ex":
          self.conv1 = nn.Conv2d(3,8,11,4)
          self.mp1 = nn.MaxPool2d(5, 2)
          self.conv2 = nn.Conv2d(8,16,5,1,2)
          self.mp2 = nn.MaxPool2d(5, 2)
          self.conv3 = nn.Conv2d(16,32,3,1,1)
          self.conv4 = nn.Conv2d(32,64,3,1,1)
          self.mp3 = nn.MaxPool2d(5, 2)        
          self.fc1 = nn.Linear(2816, 2048)
          self.fc2 = nn.Linear(2048, 1024)
          self.fc3 = nn.Linear(1024,8)
          nn.init.xavier_normal_(self.conv1.weight)
          nn.init.xavier_normal_(self.conv2.weight)
          nn.init.xavier_normal_(self.conv3.weight)
          nn.init.xavier_normal_(self.conv4.weight)
          nn.init.xavier_normal_(self.fc1.weight)
          nn.init.xavier_normal_(self.fc2.weight)
          nn.init.xavier_normal_(self.fc3.weight)
        elif self.arch[0:11] == "pytorch_adv":
            self.conv1 = nn.Conv2d(3, 4, 5)
            self.conv2 = nn.Conv2d(4, 8, 5)
            self.conv3 = nn.Conv2d(8, 16, 5)
            self.pool = nn.MaxPool2d(2, 2)
            self.drop = nn.Dropout()
            self.fc1 = nn.Linear(19968, 4096)
            self.fc2 = nn.Linear(4096, 2048)
            self.fc3 = nn.Linear(2048, 512)
            self.fc4 = nn.Linear(512, 8)
            nn.init.xavier_normal_(self.conv1.weight)
            nn.init.xavier_normal_(self.conv2.weight)
            nn.init.xavier_normal_(self.conv3.weight)
            nn.init.xavier_normal_(self.fc1.weight)
            nn.init.xavier_normal_(self.fc2.weight)
            nn.init.xavier_normal_(self.fc3.weight)
        
    def forward(self, x):
        if self.arch[0:10] == "pytorch_ex":
          x = F.rrelu(self.mp1(self.conv1(x)))
          x = F.rrelu(self.mp2(self.conv2(x)))
          x = self.mp3(self.conv4(self.conv3(x)))
          x = x.view(int(x.size(0)), -1)
          x = F.dropout(F.rrelu(self.fc1(x)))
          x = F.rrelu(self.fc2(x))
          x = self.fc3(x)
        elif self.arch[0:11] == "pytorch_adv":
            x = self.pool(F.leaky_relu(self.conv1(x)))
            x = self.pool(F.leaky_relu(self.conv2(x)))
            x = self.pool(F.leaky_relu(self.conv3(x)))
            x = x.flatten(1)            
            x = self.drop(F.leaky_relu(self.fc1(x)))
            x = self.drop(F.leaky_relu(self.fc2(x)))
            x = F.leaky_relu(self.fc3(x))
            x = self.fc4(x)
        return x

In [0]:
import pandas as pd
import numpy as np
import datetime

def preds_to_csv(y,header=""):
    y = y.astype(int)
    df = pd.DataFrame({'Category': y})
    df.index += 1  # Ensures that the index starts at 1. 
    df.to_csv(str(header+"_"+str(datetime.datetime.now().time())+'_submission.csv'),index_label='Id')
    print("saved predictions")

In [0]:
!ls

# 4a

## Model 1

In [0]:
is_train = True
is_key_frame = True
arch = "pytorch_adv_64key"
params = {'batch_size': 64,'shuffle': True,'num_workers': 4}
hyperparams = {"num_epochs":10,"criterion":nn.CrossEntropyLoss()}
model = NeuralNet(arch).to(device)
hyperparams["optimizer"] = torch.optim.Adam(model.parameters())
models_data.append(modelling(model,is_train,is_key_frame,arch,params,hyperparams))
data_track(models_data,save=True)

In [0]:
#TESTING
_,val_loader,_ = load_preprocess_data(is_train=False,
                                         is_key_frame=True,
                                         params=models_data[0][3])
key_acc,key_predicted_list,key_groundtruth_list = get_accuracy(model, val_loader)
key_pairs = move_val_acc(key_predicted_list, key_groundtruth_list)
key_pairs

## Model 2

In [0]:
models_data = data_track(save=False)
is_train = True
is_key_frame = True
arch = "pytorch_ex_64sgd"
params = {'batch_size': 64,'shuffle': True,'num_workers': 4}
hyperparams = {"num_epochs":10,"criterion":nn.CrossEntropyLoss()}
model = NeuralNet(arch).to(device)
hyperparams["optimizer"] = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
models_data.append(modelling(model,is_train,is_key_frame,arch,params,hyperparams))
data_track(models_data,save=True)

In [0]:
#TESTING
_,val_loader,_ = load_preprocess_data(is_train=False,
                                         is_key_frame=True,
                                         params=models_data[-1][3])
key_acc,key_predicted_list,key_groundtruth_list = get_accuracy(model, val_loader)
key_pairs = move_val_acc(key_predicted_list, key_groundtruth_list)
key_pairs

In [0]:
[m[2] for m in models_data]

## Model 3

In [0]:
models_data = data_track(save=False)
is_train = True
is_key_frame = True
arch = "pytorch_ex_128adam"
params = {'batch_size': 128,'shuffle': True,'num_workers': 4}
hyperparams = {"num_epochs":15,"criterion":nn.CrossEntropyLoss()}
model = NeuralNet(arch).to(device)
hyperparams["optimizer"] = torch.optim.Adam(model.parameters())
models_data.append(modelling(model,is_train,is_key_frame,arch,params,hyperparams))
data_track(models_data,save=True)

In [0]:
#TESTING
_,val_loader,_ = load_preprocess_data(is_train=False,
                                         is_key_frame=True,
                                         params=models_data[-1][3])
key_acc,key_predicted_list,key_groundtruth_list = get_accuracy(model, val_loader)
key_pairs = move_val_acc(key_predicted_list, key_groundtruth_list)
key_pairs

In [0]:
[m[2] for m in models_data]

# 4b

In [0]:
for i in range(len(models_data)):
    plot_loss(models_data[i][2],models_data[i][0],models_data[i][1])

# 4c

In [0]:
[m[2] for m in models_data]

In [0]:
sum_model = NeuralNet(models_data[1][2]).to(device)
sum_model.load_state_dict(torch.load(os.getcwd()+"/"+models_data[1][2]))
sum_model.eval()
print(sum_model)
print(summary(sum_model, input_size=(3, 224, 448)))

In [0]:
l1=(6*220*444)
l2=(6*110*222)
l3=(16*106*218)
l4=(16*53*109)
l5=512
l6=128
l7=9
num=(l1*l2)+(l2*l3)+(l3*l4)+(l4*l5)+(l5*l6)+(l6*l7)
print("number of parameters in comparable fully-connected layers NN:"
      ,num)


# 4d

In [0]:
models_data = data_track(save=False)
is_train = True
is_key_frame = False
arch = "pytorch_ex_64sgd_random"
params = {'batch_size': 64,'shuffle': True,'num_workers': 4}
hyperparams = {"num_epochs": 2,"criterion":nn.CrossEntropyLoss()}
model = NeuralNet(arch).to(device)
hyperparams["optimizer"] = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
mod_data = modelling(model,is_train,is_key_frame,arch,params,hyperparams)

In [0]:
models_data.append(mod_data)
data_track(models_data,save=True)
plot_loss(models_data[3][2],models_data[3][0],models_data[3][1])

# 4e

In [0]:
models_data = data_track(save=False)
[m[2] for m in models_data]

In [0]:
key_model = NeuralNet(models_data[1][2]).to(device)
key_model.load_state_dict(torch.load(os.getcwd()+"/"+models_data[1][2]))
key_model.eval()
_,keyval_loader,_ = load_preprocess_data(is_train=False,
                                         is_key_frame=True,
                                         params=models_data[1][3])
rand_model = NeuralNet(models_data[3][2]).to(device)
rand_model.load_state_dict(torch.load(os.getcwd()+"/"+models_data[3][2]))
rand_model.eval()
_,randval_loader,_ = load_preprocess_data(is_train=False,
                                          is_key_frame=False,
                                          params=models_data[3][3])

In [0]:
key_acc,key_predicted_list,key_groundtruth_list = get_accuracy(key_model, keyval_loader)
rand_acc,rand_predicted_list,rand_groundtruth_list = get_accuracy(rand_model, randval_loader)

In [0]:
key_pairs = move_val_acc(key_predicted_list, key_groundtruth_list)
rand_pairs = move_val_acc(rand_predicted_list, rand_groundtruth_list)

In [0]:
accs = [["key frame"]+[p[0] for p in key_pairs]+[key_acc],["random frame"]+[p[0] for p in rand_pairs]+[rand_acc]]
cols = ["frame type",'reach','squat','inline','lunge','hamstrings','stretch','deadbug','pushup',"overall"]
df = pd.DataFrame(accs,columns=cols)
print("accuracies")
df

# 4f

In [0]:
cm = confusion_matrix(key_groundtruth_list, 
                 key_predicted_list)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
fig, ax = plt.subplots()
classes = ['reach','squat','inline','lunge','hamstrings','stretch','deadbug','pushup']
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       xticklabels=classes, yticklabels=classes,
       title="confusion matrix for key frame model",
       ylabel='True label',
       xlabel='Predicted label');
plt.xticks(rotation=90);